In [0]:
#  Defining the paths, these volumes pointing towards the Data Lake container(external location) 
stage_table = 'autoloader_catalog.orders_schema.orders_stage'
archive_path = '/Volumes/autoloader_catalog/default/external_source/archived_files/'
temp_loc_path = '/Volumes/autoloader_catalog/default/external_source/temp_location/'
checkpoint_path = '/Volumes/autoloader_catalog/default/external_source/checkpoint/'

In [0]:
# reading the data from the temp location
df = spark.read.format('csv')\
        .option('inferSchema',True)\
        .option('header',True)\
        .load(temp_loc_path)

In [0]:
# writing (over writing) each time to the stage table (managed delta table) 
df.write.mode('overwrite')\
        .format('delta')\
        .option('header',True)\
        .option('overwriteSchema',True)\
        .saveAsTable(stage_table)

# writing the data to the archive for the backup
df.write.mode('append')\
        .format('csv')\
        .option('header',True)\
        .save(archive_path)


In [0]:
# Deleting the current daily order data from the temp location
files = dbutils.fs.ls('/Volumes/autoloader_catalog/default/external_source/temp_location/')

for file in files:
    dbutils.fs.rm(file.path, recurse=True)

# Deleting the complete checkpoint details except the folder
items = dbutils.fs.ls(checkpoint_path)

for item in dbutils.fs.ls(checkpoint_path):
    dbutils.fs.rm(item.path, recurse=True)

# Deleting files in the daily orders from the current day
dfiles = dbutils.fs.ls('/Volumes/autoloader_catalog/default/external_source/daily_order_files/')

for file in dfiles:
    dbutils.fs.rm(file.path, recurse=True)
    

##### for the fresh start deleting the data in the archive path
a_files = dbutils.fs.ls(archive_path)

for file in a_files:
    dbutils.fs.rm(file.path, recurse=True)


